# Лабораторная работа 1. Подсчет ошибки распознавания

WER (Word Error Rate) является метрикой для оценки качества систем распознавания речи. Она показывает процент ошибочных слов в гипотезе распознавания по сравнению с эталонным текстом. WER учитывает три типа ошибок: вставки, удаления и замены. 
$$ WER = {I + D + S \over D + S + C} $$
где I, D, S - количество втавок, удалений и замен, соответственно. C - количество правильно распознанных слов

Лабораторная работа состоит из трех частей. Первая часть (функция подсчета WER) обязательная, остальные дополнительные. Всего за работу можно получить максимум 20 баллов. 4 за сдачу в срок и 16 за задания: 
* функция подсчета WER (тест 1.a, 1.b) - 8 баллов
* функция подсчета WER и ошибки пунктуации (тест 2.a) - 4 балла
* функция подсчета SA-WER (тест 3.а) - 4 балла

# 1. Word Error Rate (8 баллов)

## 1.a. подсчет WER 


Функция должна принимать две строки в качестве входных данных: эталонный текст и распознанный текст. Эталонный текст - это то, что произносится в аудиозаписи, а гипотеза распознавания - это текст, полученный от системы распознавания речи. Для корректного вычисления ошибки распознавания необходимо удалить все символы пунктуации и привести все слова к нижнему регистру.



In [44]:

def calculate_wer(reference_text: str, recognized_text: str) -> float:
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    
    import string
    reference_words = reference_text.lower().translate(str.maketrans('', '', string.punctuation)).split()
    recognized_words = recognized_text.lower().translate(str.maketrans('', '', string.punctuation)).split()

    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]

    
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    

    # Заполнение матрицы расстояний методом динамического программирования
    # нашел тут инструкцию https://habr.com/ru/articles/676858/

    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            cost = 0 if reference_words[i - 1] == recognized_words[j - 1] else 1 # если слова одинаковы то штраф == 0
            
            distance_matrix[i][j] = min(
                distance_matrix[i - 1][j] + 1,  # смотрим выше на одну строку
                distance_matrix[i][j - 1] + 1,  # смотрим на предыдущий столбец
                distance_matrix[i - 1][j - 1] + cost  # смотрим на предыдущйю ячейку по диагонали (алг Вагнера — Фишера может приснится только в кошмаре)
            )
            
            #from tabulate import tabulate
            #print(tabulate(distance_matrix, headers=reference_words, showindex=['',*recognized_words] ))
    # Расчет WER (в процентах)
    
    wer = distance_matrix[len(reference_words)][len(recognized_words)] / len(reference_words) * 100 if reference_words else 0
    return wer

wer = calculate_wer('Я ел солонину', 'Я ел слона')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 33.33%


In [16]:
def assert_wer(ref, hyp, ideal_wer):
    wer = calculate_wer(ref, hyp)
    assert round(wer, 2) == round(ideal_wer, 2), f"for '{hyp=}' and '{ref=}' {ideal_wer=}, calculate_wer {wer=}"
    
def test_wer():
    assert_wer('привет студент', 'привет студент', 0)
    assert_wer('привет! Студент.', 'Привет, студент?', 0)
    assert_wer('привет студент', 'студент', 50)
    assert_wer('привет студент', '', 100)
    assert_wer('привет студент', 'студент привет', 100)
    assert_wer('привет', 'привет студент', 100)
    assert_wer('привет студент привет как дела', 'студент привет', 60)
    assert_wer('привет студент привет как дела', 'привет как дела', 40)
    assert_wer('привет студент привет как дела ', 'привет студент дела ', 40)
    assert_wer('привет студент привет как дела '*100, 'привет студент дела '*100, 40)

    print(f"Test 1.a passed")
    
test_wer() 
    

Test 1.a passed


## 1.b. Построение выравнивания
Реализованная в части 1.a. функция выдает только суммарное значение ошибки распознавания, не давая понимания, в чем состоят основные проблемы распознавания. 

Значение WER получается из трех видов ошибок: 
* вставка (insertion)
* удаление (deletion)
* замена (substitution)

Каждый тип ошибок имеет свое значение и указывает на определенные недостатки системы. Например, большое количество вставок означает, что ASR слышит речь там где ее нет. А большое количество удалений показывает, что целевая речь пропускается и не транскрибируется. 

Кроме числовых значений каждой ошибки, для анализа результатов работы системы может пригодиться выравнивание эталонной текстовки и гипотезы распознавания относительно друг друга. 

пример выравнивания: 

```
>>> tabulate(ali)

Я сегодня  ***   учуcь  в  универе
Я    с    завтра учусь *** универе  
C    S      I      C    D    C    
```

Реализуйте функцию, которая кроме числового значения WER возвращает выравнивание, а также значения каждого типа ошибок распознавания (вставки, удаления, замены).

In [20]:
!pip install tabulate
from tabulate import tabulate
# используйте tabulate для отладки


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:

def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
    
    
    import string
    reference_words = reference_text.lower().translate(str.maketrans('', '', string.punctuation)).split()
    recognized_words = recognized_text.lower().translate(str.maketrans('', '', string.punctuation)).split()

    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]

    
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            cost = 0 if reference_words[i - 1] == recognized_words[j - 1] else 1 # если слова одинаковы то штраф == 0
            
            distance_matrix[i][j] = min(
                distance_matrix[i - 1][j] + 1,  # смотрим выше на одну строку (удаление)
                distance_matrix[i][j - 1] + 1,  # смотрим на предыдущий столбец (вставка)
                distance_matrix[i - 1][j - 1] + cost  # смотрим на предыдущйю ячейку по диагонали (замена)
            )
            
            #print(distance_matrix)
    # Расчет WER (в процентах)
    
    wer = distance_matrix[len(reference_words)][len(recognized_words)] / len(reference_words) * 100 if reference_words else 0

    # используя distance_matrix восстановите путь (набор операций), который соответстует найденому WER 
    #TODO

    # Восстановление пути по матрице расстояний
    # нашел тут иструкцию https://habr.com/ru/articles/117063/

    i, j = len(reference_words), len(recognized_words)
    ali_ref, ali_hyp, ali_ops = [], [], []  # алигнмент для референса, гипотезы и операций
    correct, deletion, insertion, substitution = 0, 0, 0, 0

    while i > 0 or j > 0:
        if i > 0 and j > 0 and distance_matrix[i][j] == distance_matrix[i - 1][j - 1] + (1 if reference_words[i - 1] != recognized_words[j - 1] else 0):
            ali_ref.append(reference_words[i - 1])
            ali_hyp.append(recognized_words[j - 1])
            if reference_words[i - 1] == recognized_words[j - 1]:
                ali_ops.append("C")  # Correct
                correct += 1
            else:
                ali_ops.append("S")  # Substitution
                substitution += 1
            i -= 1
            j -= 1
        elif i > 0 and distance_matrix[i][j] == distance_matrix[i - 1][j] + 1:
            ali_ref.append(reference_words[i - 1])
            ali_hyp.append("***")  # Вставка в гипотезу
            ali_ops.append("D")  # Deletion
            deletion += 1
            i -= 1
        else:
            ali_ref.append("***")
            ali_hyp.append(recognized_words[j - 1])
            ali_ops.append("I")  # Insertion
            insertion += 1
            j -= 1

    # Переворачиваем алигнмент, так как шли от конца к началу
    ali_ref.reverse()
    ali_hyp.reverse()
    ali_ops.reverse()

    ali = [ali_ref, ali_hyp, ali_ops]

    # ali[0]=  разбитый по словам референс. Втавки отабражаются в эталонном выравнивании с помощью "***"
    # ali[1] = разбитая по словам гипотеза.
    # ali[2] = аннотация 
    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"
    
    return {"wer" : wer,
            "cor": correct, 
            "del": deletion,
            "ins": insertion,
            "sub": substitution,
            "ali": ali}


In [18]:
def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
    return {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]}

In [22]:
def assert_wer_with_alignment(ref, hyp, ideal_report):
    report = calculate_wer_with_alignment(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_with_alignment():
    assert_wer_with_alignment('привет студент', 'привет студент',  {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]})
    assert_wer_with_alignment('привет студент', 'студент', {
            "wer" : 50,
            "cor": 1, 
            "del": 1,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})
    assert_wer_with_alignment('привет', 'привет студент', {
            "wer" : 100,
            "cor": 1, 
            "del": 0,
            "ins": 1,
            "sub": 0,
            "ali": [["привет", "***"],["привет", "студент"],['C', 'I']]})
    assert_wer_with_alignment('привет студент', 'пока студент',  {
            "wer" : 50,
            "cor": 1, 
            "del": 0,
            "ins": 0,
            "sub": 1,
            "ali": [["привет", "студент"],["пока", "студент"],['S', 'C']]})

    print(f"Test 1.b passed")
    
test_wer_with_alignment() 

Test 1.b passed


# 2. WER с пунктуацией (4 балла)
Попробуйте модифицировать WER таким образом, чтобы получившаяся метрика учитавала ошибки расстановки знаков препинания. 

Для этого надо ввести ограничение в алгоритм подсчета distance_matrix таким образом, чтобы запретить делать замену знака препинания на слово и наоборот.

Пример выравнивания 
```
Я сегодня  .   ***   ***  А ты  
Я    с    *** завтра  ?   А ты  
C    S    D_p   I    I_p  C  C    
```
Здесь суффикс _p в аннотации к ошибкам означает **ошибки пунктуации**


Задание: 
Напишите функцию, которая кроме стандартного WER считает дополнительно RichTranscriptErrorRate (RTER) по формуле

$$ RTER = {I_p + D_p + S_p \over D_p + S_p + C_p} $$


In [132]:
def calculate_wer_per(reference_text: str, recognized_text: str):
    
    import string

    # Списки знаков препинания
    punctuation = set(string.punctuation)
    
    # Приведение к нижнему регистру и разделение на слова (включая знаки препинания)
    import re
    reference_words  = re.findall(r'\w+|[^\w\s]', reference_text.lower(), re.UNICODE)
    recognized_words = re.findall(r'\w+|[^\w\s]', recognized_text.lower(), re.UNICODE)

    print(reference_words)
    # Инициализация матрицы для подсчета расстояния
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    
    # Наполнение первой строки и первого столбца
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j
    
    # Переменные для подсчета ошибок
    substitution_p, deletion_p, insertion_p, correct_p = 0, 0, 0, 0

    dict_p = {'correct_p': 0,                   # попоробуем другой способ подсчета
                       'deletion_p':0 ,
                       'insertion_p':0,
                       'substitution_p':0
                       }


    # Заполнение матрицы с учетом запрещения замены слова на знак препинания и наоборот
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            ref_is_punct = reference_words[i - 1] in punctuation
            rec_is_punct = recognized_words[j - 1] in punctuation

            cost = 0          
            if reference_words[i - 1] == recognized_words[j - 1]:
                #cost = 0  # Совпадение
                if ref_is_punct:   # два одинаковых знака препинания
                    cost = 0  # Совпадение
                    correct_p += 1  # Правильный знак препинания
                    dict_p['correct_p']+= 1 # Правильный знак препинания

                    #print('correct_p ', correct_p ,reference_words[i - 1],  recognized_words[j - 1] )

            elif ref_is_punct != rec_is_punct: # токены не совпадают и один из них знак препинания
                cost = 2  # Запрещаем замену слова на знак препинания или наоборот
                if ref_is_punct:
                    
                    deletion_p += 1
                    #print('deletion_p', deletion_p ,reference_words[i - 1],  recognized_words[j - 1] )
                else:
                    
                    insertion_p += 1
                    #print('insertion_p', insertion_p ,reference_words[i - 1],  recognized_words[j - 1] )
            else:
                #cost = 1  # Замена одного слова на другое
                if ref_is_punct and rec_is_punct:
                    cost = 1  # Замена одного слова на другое
                    substitution_p += 1
                    dict_p['substitution_p']+= 1
                    #print('substitution_p', substitution_p ,reference_words[i - 1],  recognized_words[j - 1] )

            list_for_matrix = [distance_matrix[i - 1][j] + 1,  # удаление
                                distance_matrix[i][j - 1] + 1,  # вставка
                                distance_matrix[i - 1][j - 1] + cost  # замена
                                ]


            distance_matrix[i][j] = min(list_for_matrix)
            
            '''
            min_index = list_for_matrix.index(distance_matrix[i][j])
            
            if (ref_is_punct or rec_is_punct) and reference_words[i - 1] != recognized_words[j - 1]:  #если есть один или два знака препинания и они не совпадают
                if min_index == 0:
                    dict_p['deletion_p']+= 1
                if min_index == 1:
                    dict_p['insertion_p']+= 1

            #print('для пары_',reference_words[i - 1], recognized_words[j - 1], 'действие № ', min_index  )
                    
    #substitution_p, deletion_p, insertion_p, correct_p = dict_p['substitution_p'], dict_p['deletion_p'], dict_p['insertion_p'], dict_p['correct_p']
    #print( [substitution_p, deletion_p, insertion_p, correct_p] == [dict_p['substitution_p'], dict_p['deletion_p'], dict_p['insertion_p'], dict_p['correct_p']])
        '''
    # Расчет WER (в процентах)
    wer = distance_matrix[len(reference_words)][len(recognized_words)] / len(reference_words) * 100 if reference_words else 0

    # Вычисление PER по новой формуле
    per = (insertion_p + deletion_p + substitution_p) / (deletion_p + substitution_p + correct_p) * 100 if (deletion_p + substitution_p + correct_p) > 0 else 0

  

    return {"wer" : calculate_wer(reference_text, recognized_text),
            "per": per }
   


In [134]:
calculate_wer_per('привет студент.', 'студент.')

['привет', 'студент', '.']


{'wer': 50.0, 'per': 150.0}

In [112]:
def assert_wer_per(ref, hyp, ideal_report):
    report = calculate_wer_per(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_per():
    assert_wer_per('привет студент.', 'привет студент',  {
            "wer" : 0,
            "per": 100})
    assert_wer_per('привет студент.', 'студент.', {
            "wer" : 50,
            "per": 0,})
    assert_wer_per('привет студент.', 'привет. студент',  {
            "wer" : 0,
            "per": 200})
    assert_wer_per('привет студент.', '.студент?', {
            "wer" : 50,
            "per": 200, })

    print(f"Test 2 passed")
    
test_wer_per() 

['привет', 'студент', '.']
deletion_p 0 . привет
deletion_p 0 . студент
['привет', 'студент', '.']
insertion_p 0 привет .
insertion_p 0 студент .
deletion_p 0 . студент


AssertionError: for 'hyp='студент.'' and 'ref='привет студент.'' ideal_report={'wer': 50, 'per': 0}, calculate_wer report={'wer': 50.0, 'per': 150.0}

# 3. Speaker-attributed Word Error Rate (4 балла)

В задаче распознавания диалоговых данных, когда говорят два диктора, важно не только распознать правильно каждое слово, но и отнести его к правильному диктору. Чаще всего такой результат получается с помощью комбинации двух независимых систем: распознавания речи и диаризация. 

При подсчете ошибки распознавания диалоговых систем в формулу WER добавляется еще один тип ошибки - S_I (speaker incorrect).

$$ SA{\text -}WER = \min{I + D + S + S_I \over D + S + C + S_I} $$

Кроме подсчета самой ошибки, sa-wer решает еще одну задачку - поиск маппинга из эталонных названий спикеров (например, имен) в предсказанные (чаще всего Idшники). Это необходимо, тк система диаризации не знает, какие названия у спикеров в эталоне. При подсчете SA-WER проверяются все возможные мапинги спикеров и выбирается тот, который соответствует минимальному значению ошибки. 



In [139]:
import itertools

def calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers):
    # В отличие от прошлых функций на вход sawer подаются уже разбитые на слова произнесения
    # Кроме списка слов, дополнительно передается список меток спикеров
    assert isinstance(reference_text, list)
    assert isinstance(recognized_text, list)
    assert len(reference_text) == len(reference_speakers)
    assert len(recognized_text) == len(recognized_speakers)
    
    #При подсчете ошибки распознавания диалоговых систем в формулу WER добавляется еще один тип ошибки - S_I (speaker incorrect) $$ SA{\text -}WER = \min{I + D + S + S_I \over D + S + C + S_I} $$
    # TODO  посчитайте sa-wer с учетом мапинга спикеров
    # для этого посчитайте значение ошибки для каждого варианта мапинга меток дикторов 
    # и выберете тот, который соответствует минимальному SA-WER
# Инициализируем переменные для хранения минимального SA-WER и соответствующего выравнивания
    min_sawer = float('inf')
    best_ali = []
    
    # Все возможные варианты маппинга спикеров
    possible_mappings = list(itertools.permutations(set(recognized_speakers)))

    for mapping in possible_mappings:
        # Преобразуем метки распознанных спикеров в соответствии с текущей перестановкой
        mapped_speakers = [mapping[recognized_speakers.index(speaker)] for speaker in recognized_speakers]

        # Считаем обычные ошибки WER (I, D, S)
        distance_matrix = [[0] * (len(recognized_text) + 1) for _ in range(len(reference_text) + 1)]
        for i in range(1, len(reference_text) + 1):
            for j in range(1, len(recognized_text) + 1):
                cost = 0 if reference_text[i - 1] == recognized_text[j - 1] else 1
                distance_matrix[i][j] = min(
                    distance_matrix[i - 1][j] + 1,  # Удаление
                    distance_matrix[i][j - 1] + 1,  # Вставка
                    distance_matrix[i - 1][j - 1] + cost  # Замена
                )
        
        # Считаем speaker incorrect (S_I)
        speaker_errors = sum(1 for ref_sp, rec_sp in zip(reference_speakers, mapped_speakers) if ref_sp != rec_sp)

        # Общие ошибки для вычисления SA-WER
        I = len(recognized_text) - len(reference_text)  # Вставки
        D = len(reference_text) - len(recognized_text)  # Удаления
        S = distance_matrix[len(reference_text)][len(recognized_text)]  # Замены
        SI = speaker_errors  # Ошибки спикеров

        # Считаем SA-WER
        C = len(reference_text) - S - D  # Верные совпадения
        sawer = (I + D + S + SI) / (D + S + C + SI) * 100
        
        # Сохраняем минимальный SA-WER и соответствующее выравнивание
        if sawer < min_sawer:
            min_sawer = sawer
            best_ali = {
                "reference": reference_text,
                "recognized": recognized_text,
                "reference_speakers": reference_speakers,
                "mapped_speakers": mapped_speakers
            }
    
    return {"sawer": min_sawer, "ali": best_ali}


def assert_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers, ideal_report):
    report = calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers)
    for k, v in ideal_report.items():
        assert v == report[k]

    


In [137]:
from itertools import permutations

def calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers):
    assert isinstance(reference_text, list)
    assert isinstance(recognized_text, list)
    assert len(reference_text) == len(reference_speakers)
    assert len(recognized_text) == len(recognized_speakers)
    
    # Расстояние Левенштейна для WER (подсчет ошибок вставки, удаления и замены)
    def levenshtein_distance(ref, hyp):
        dp = [[0] * (len(hyp) + 1) for _ in range(len(ref) + 1)]
        for i in range(len(ref) + 1):
            dp[i][0] = i
        for j in range(len(hyp) + 1):
            dp[0][j] = j
        
        for i in range(1, len(ref) + 1):
            for j in range(1, len(hyp) + 1):
                if ref[i-1] == hyp[j-1]:
                    cost = 0
                else:
                    cost = 1
                dp[i][j] = min(dp[i-1][j] + 1, dp[i][j-1] + 1, dp[i-1][j-1] + cost)
        
        return dp[len(ref)][len(hyp)], dp

    # Подсчет ошибок спикеров для каждого маппинга
    def calculate_speaker_error(reference_speakers, recognized_speakers, mapping):
        speaker_errors = 0
        for i, ref_speaker in enumerate(reference_speakers):
            if ref_speaker != mapping[recognized_speakers[i]]:
                speaker_errors += 1
        return speaker_errors

    # Все возможные маппинги спикеров
    unique_speakers_ref = sorted(set(reference_speakers))
    unique_speakers_rec = sorted(set(recognized_speakers))
    
    # Найдем все возможные соответствия дикторов
    all_mappings = list(permutations(unique_speakers_rec, len(unique_speakers_ref)))

    min_sawer = float('inf')
    best_ali = []

    # Расчет для каждого маппинга
    for mapping in all_mappings:
        mapping_dict = {rec_speaker: ref_speaker for rec_speaker, ref_speaker in zip(mapping, unique_speakers_ref)}
        
        # Расстояние Левенштейна
        wer, dp_matrix = levenshtein_distance(reference_text, recognized_text)
        
        # Подсчет ошибок спикеров
        speaker_error = calculate_speaker_error(reference_speakers, recognized_speakers, mapping_dict)
        
        # Подсчет общего количества правильных, удаленных и замененных слов
        C = len(reference_text) - wer
        D = dp_matrix[len(reference_text)][0]
        S = dp_matrix[len(reference_text)][len(recognized_text)] - C
        
        # Подсчет SA-WER
        sawer = (S + D + speaker_error) / (S + D + C + speaker_error)
        
        if sawer < min_sawer:
            min_sawer = sawer
            best_ali = [reference_text, recognized_text, mapping_dict]

    return {"sawer": min_sawer * 100, "ali": best_ali}

In [140]:
def assert_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers, ideal_report):
    report = calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers)
    for k, v in ideal_report.items():
        assert v == report[k]

    
def test_sawer():
    assert_sawer(['привет', 'студент'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 0})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 0})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет'], [1],  {
            "sawer" : 50})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [1, 0],  {
            "sawer" : 100})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 100})

    print(f"Test 3 passed")
    
test_sawer()

AssertionError: 